# Initialization

### Package used

In [1]:
Pkg.add("Graphs")
Pkg.add("LightGraphs")

INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of Graphs
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of LightGraphs
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [2]:
Pkg.update()

INFO: Updating METADATA...
INFO: Updating cache of Requires...
INFO: Updating cache of DataFrames...
INFO: Updating cache of Distributions...
INFO: Updating GeneratedTypes...
INFO: Updating GeneratedTables...
INFO: Updating TSne...
INFO: Updating GraphPlot...
INFO: Updating ComputeFramework...
INFO: Updating ELM...
INFO: Updating PlotlyJS...
INFO: Updating ROC...
INFO: Computing changes...
INFO: Upgrading DataFrames: v0.7.6 => v0.7.7
INFO: Upgrading Distributions: v0.10.0 => v0.10.1
INFO: Building Rmath


In [3]:
using Graphs
using LightGraphs

### Auxiliary Functions

In [22]:
function CM2EL{T <: Union{Real, Bool}}(M::Array{T, 2}) # convert a connection (adjacency) matrix to an edge list
    n = sqrt(length(M))    
    
    if n != round(n)
        println("Matrix needs to be square!")
        return []
    end
    
    if eltype(M) != Bool
        AM = (M .!= 0.0) # weights can be negative in some cases, so any non-zero element is a connection
    else
        AM = M
    end
    
    n = convert(Int64, n)
    N = div(sum(AM), 2) # maximum possible number of edges in graph
    z = collect(1:n)
    E = Array(Int64, N, 2)
    c = 0

    for i = 1:(n-1)
        for j = (i+1):n
            if AM[i,j]
                c += 1
                E[c,1] = i
                E[c,2] = j                
            end
        end
    end
    
    return E[1:c,:]
end

CM2EL (generic function with 1 method)

In [5]:
function EL2CM{T <: Real}(E::Array{Int64, 2}, n::Int64, w::Array{T, 1} = [0.0]) # convert an edge list to a connection (adjacency) matrix
    if n < 2
        println("Trivial case. Aborting...")
        return -ones(1,1)
    end
    
    if n < maximum(E)
        println("Number of nodes is too small and/or there is a mistake in the edge list...")
        return -ones(1,1)
    end
    
    nwp = (length(w) == 1) # no weights were provided
    ne = size(E, 1) # number of edges
        
    if nwp
        M = Array(Bool, n, n)
        M[:] = false
    else
        if length(w) != ne
            println("Number of weights must be equal to the number of edges!")
            return -ones(1,1)
        end
        
        M = zeros(T, n, n)
    end    

    for i = 1:ne
        x, y = E[i,:]
        
        if nwp
            M[x,y] = true
            M[y,x] = true
        else
            M[x,y] = w[i]
            M[y,x] = w[i]
        end
    end
    
    return M
end

EL2CM (generic function with 2 methods)

In [6]:
function G2LG{T <: GenericGraph}(G::T)
    n = num_vertices(G)

    if Graphs.is_directed(G)
        g = LightGraphs.DiGraph(n)
    else
        g = LightGraphs.Graph(n)
    end
    
    AM = Graphs.adjacency_matrix(G)
    z = collect(1:n)
    
    for i = 1:n        
        X = z[AM[i,:][:]]
        
        for x in X
            LightGraphs.add_edge!(g, i, x)
        end
    end
    
    return g
end

G2LG (generic function with 1 method)

In [7]:
function LG2G(g::LightGraphs.Graph) # UNdirected graph created with LightGraphs
    n = nv(g)
    G = simple_graph(n, is_directed=false)
    E = collect(LightGraphs.edges(g))
    N = length(E)
        
    for i = 1:N      
        x, y = E[i]
        Graphs.add_edge!(G, x, y)
    end
    
    return G
end

LG2G (generic function with 1 method)

In [8]:
function LG2G(g::LightGraphs.DiGraph) # directed graph created with LightGraphs
    n = nv(g)
    G = simple_graph(n, is_directed=true)
    E = collect(LightGraphs.edges(g))
    N = length(E)
        
    for i = 1:N      
        x, y = E[i]
        Graphs.add_edge!(G, x, y)
    end
    
    return G
end

LG2G (generic function with 2 methods)

In [9]:
include("normalize.jl")

normalize (generic function with 4 methods)

### Dataset Creation

In [10]:
g = simple_graph(7, is_directed=false)

Undirected Graph (7 vertices, 0 edges)

In [11]:
Graphs.add_edge!(g, 1, 2)
Graphs.add_edge!(g, 1, 3)
Graphs.add_edge!(g, 1, 5)
Graphs.add_edge!(g, 1, 7)
Graphs.add_edge!(g, 2, 3)
Graphs.add_edge!(g, 2, 5)
Graphs.add_edge!(g, 2, 6)
Graphs.add_edge!(g, 2, 7)
Graphs.add_edge!(g, 3, 4)
Graphs.add_edge!(g, 3, 6)
Graphs.add_edge!(g, 3, 7)
Graphs.add_edge!(g, 4, 6)
Graphs.add_edge!(g, 4, 7)
Graphs.add_edge!(g, 5, 6)
Graphs.add_edge!(g, 6, 7)

edge [15]: 6 -- 7

In [12]:
w = [0.00956345, 0.952837, 0.775323, 0.0875062, 0.546019, 0.573757, 0.43756, 0.665684, 0.445205, 0.248718, 0.977219, 0.732508, 0.149959, 0.284119, 0.990848]

15-element Array{Float64,1}:
 0.00956345
 0.952837  
 0.775323  
 0.0875062 
 0.546019  
 0.573757  
 0.43756   
 0.665684  
 0.445205  
 0.248718  
 0.977219  
 0.732508  
 0.149959  
 0.284119  
 0.990848  

### Loading Conventional Dataset and Obtaining the Features Graph

In [13]:
ONP = readcsv("D:\\data\\OnlineNewsPopularity\\OnlineNewsPopularity.csv")

39645x61 Array{Any,2}:
 "url"                                                                                    …      " shares"
 "http://mashable.com/2013/01/07/amazon-instant-video-browser/"                               593         
 "http://mashable.com/2013/01/07/ap-samsung-sponsored-tweets/"                                711         
 "http://mashable.com/2013/01/07/apple-40-billion-app-downloads/"                            1500         
 "http://mashable.com/2013/01/07/astronaut-notre-dame-bcs/"                                  1200         
 "http://mashable.com/2013/01/07/att-u-verse-apps/"                                       …   505         
 "http://mashable.com/2013/01/07/beewi-smart-toys/"                                           855         
 "http://mashable.com/2013/01/07/bodymedia-armbandgets-update/"                               556         
 "http://mashable.com/2013/01/07/canon-poweshot-n/"                                           891         
 "http://masha

In [14]:
nd = size(ONP, 2) - 2 # number of dimensions (without the target variable or the index)

59

In [15]:
ONP = normalize(map(Float64,ONP[2:end,2:end]), "stat")

39644x60 Array{Float64,2}:
  1.75786   0.757438  -0.695202   …  -1.8107     0.138918   -0.241025 
  1.75786  -0.661648  -0.618786       0.837738  -0.689649   -0.230876 
  1.75786  -0.661648  -0.712183       0.837738  -0.689649   -0.163016 
  1.75786  -0.661648  -0.0329325      0.837738  -0.689649   -0.188818 
  1.75786   1.23047    1.11543       -1.56993   -0.0870549  -0.248593 
  1.75786  -0.18862   -0.37468    …  -1.054      0.257285   -0.218491 
  1.75786  -1.13468    0.877688       0.837738  -0.689649   -0.244207 
  1.75786   0.757438   0.939245       0.837738   1.51986    -0.215394 
  1.75786   0.284409  -0.954166       0.17563   -0.689649    0.0175988
  1.75786  -0.18862   -0.66973        0.837738  -0.689649   -0.230962 
  1.75786  -0.661648   1.48901    …   0.837738  -0.689649   -0.102811 
  1.75786  -0.18862   -0.763127       0.837738  -0.689649   -0.128613 
  1.75786  -0.661648  -0.578456       0.837738   3.72938    -0.221243 
  ⋮                               ⋱               

In [16]:
F = ONP[:,1:(end-1)];

In [17]:
CM = cor(F) # Correlation Matrix

59x59 Array{Float64,2}:
  1.0          -0.24032      -0.0628668    …   0.0115507   -0.00274519
 -0.24032       1.0           0.0181596       -0.146954     0.0405497 
 -0.0628668     0.0181596     1.0              0.00713597   0.0134393 
  0.00286616   -0.00531822   -0.00473669      -0.00924245  -0.00421656
  8.93253e-5   -0.00475391    0.0175118       -0.0085109   -0.0043914 
  0.00380488   -0.00541976    0.000373251  …  -0.0085723   -0.00534239
 -0.000832066  -0.0534962     0.423065         0.00944307   0.0565254 
  0.0645305    -0.0148562     0.304682         0.00896102  -0.00670937
 -0.0276359    -0.00885831    0.3426          -0.0137594    0.0633067 
  0.000935717   0.0514602     0.103699        -0.0219817    0.0552314 
  0.130465     -0.0714025     0.167789     …   0.0265856   -0.0369527 
  0.0468835    -0.00607696    0.0728448       -0.0109917    0.0235327 
  0.0544918    -0.0708153     0.0375483        0.00686095   0.0179655 
  ⋮                                        ⋱         

In [18]:
M = Array(Bool, nd, nd) # connectivity matrix, based on strong correlations

59x59 Array{Bool,2}:
  true  false   true   true  false  …  false  false  false   true  false
  true  false   true   true  false     false  false  false  false   true
 false   true   true  false  false     false   true  false  false   true
 false  false   true  false  false      true   true  false   true  false
  true   true  false   true  false     false   true  false   true   true
 false  false   true  false  false  …   true   true  false   true  false
 false  false   true   true  false      true   true  false  false   true
  true  false   true   true  false      true   true   true  false   true
  true  false   true  false  false      true   true  false  false   true
 false  false   true   true  false     false   true   true  false   true
 false  false  false   true  false  …  false   true   true  false   true
 false  false  false   true  false      true   true  false  false   true
 false   true   true   true  false     false  false   true   true  false
     ⋮                        

In [19]:
for i = 1:(nd-1)
    M[i,i] = true
    for j = (i+1):nd
        M[i,j] = M[j,i] = (abs(CM[i,j]) .> 0.7)
    end
end

In [23]:
E = CM2EL(M)

17x2 Array{Int64,2}:
  4   5
  4   6
  5   6
 15  39
 17  43
 18  41
 19  23
 20  21
 26  27
 28  30
 29  30
 37  38
 45  48
 47  49
 50  52
 53  54
 56  59

In [24]:
g2 = simple_graph(nd, is_directed=false)

Undirected Graph (59 vertices, 0 edges)

In [25]:
for i = 1:size(E, 1)
    e = E[i,:]
    Graphs.add_edge!(g2, e[1], e[2])
end

In [26]:
g2

Undirected Graph (59 vertices, 17 edges)

# Graph Analysis

### Basic Stats of Graph

In [27]:
num_vertices(g)

7

In [28]:
num_edges(g)

15

In [29]:
Graphs.vertices(g)

1:7

In [30]:
Graphs.edges(g)

15-element Array{Graphs.Edge{Int64},1}:
 edge [1]: 1 -- 2 
 edge [2]: 1 -- 3 
 edge [3]: 1 -- 5 
 edge [4]: 1 -- 7 
 edge [5]: 2 -- 3 
 edge [6]: 2 -- 5 
 edge [7]: 2 -- 6 
 edge [8]: 2 -- 7 
 edge [9]: 3 -- 4 
 edge [10]: 3 -- 6
 edge [11]: 3 -- 7
 edge [12]: 4 -- 6
 edge [13]: 4 -- 7
 edge [14]: 5 -- 6
 edge [15]: 6 -- 7

In [31]:
Graphs.is_directed(g)

false

In [32]:
out_degree(1, g)

4

In [33]:
Graphs.out_neighbors(1, g)

Graphs.TargetIterator{Graphs.GenericGraph{Int64,Graphs.Edge{Int64},UnitRange{Int64},Array{Graphs.Edge{Int64},1},Array{Array{Graphs.Edge{Int64},1},1}},Array{Graphs.Edge{Int64},1}}(Undirected Graph (7 vertices, 15 edges),[edge [1]: 1 -- 2,edge [2]: 1 -- 3,edge [3]: 1 -- 5,edge [4]: 1 -- 7])

In [34]:
n = num_vertices(g)
d = Array(Int64, n)

for i = 1:n
    d[i] = out_degree(i, g)
end

dg = maximum(d)

5

### Cycle Detection

In [35]:
test_cyclic_by_dfs(g)

true

In [36]:
test_cyclic_by_dfs(g2)

true

### Connected Components

In [37]:
Graphs.connected_components(g)

1-element Array{Array{Int64,1},1}:
 [1,2,3,5,7,6,4]

In [38]:
Graphs.connected_components(g2)

43-element Array{Array{Int64,1},1}:
 [1]    
 [2]    
 [3]    
 [4,5,6]
 [7]    
 [8]    
 [9]    
 [10]   
 [11]   
 [12]   
 [13]   
 [14]   
 [15,39]
 ⋮      
 [42]   
 [44]   
 [45,48]
 [46]   
 [47,49]
 [50,52]
 [51]   
 [53,54]
 [55]   
 [56,59]
 [57]   
 [58]   

In [39]:
CM[56, 59]

0.714527589349792

### Maximal Clique

In [40]:
Graphs.maximal_cliques(g)

5-element Array{Array{Int64,N},1}:
 [4,7,3,6]
 [2,7,3,6]
 [2,7,3,1]
 [2,5,6]  
 [2,5,1]  

### Shortest Distance / Path in a Graph

In [41]:
ww = 1 - w

15-element Array{Float64,1}:
 0.990437
 0.047163
 0.224677
 0.912494
 0.453981
 0.426243
 0.56244 
 0.334316
 0.554795
 0.751282
 0.022781
 0.267492
 0.850041
 0.715881
 0.009152

In [42]:
z = Graphs.dijkstra_shortest_paths(g, ww, 1)

Graphs.DijkstraStates{Int64,Float64,DataStructures.MutableBinaryHeap{Graphs.DijkstraHEntry{Int64,Float64},DataStructures.LessThan},Int64}([1,7,1,6,1,7,3],[1,7,1,6,1,7,3],[0.0,0.40425999999999995,0.047162999999999955,0.346588,0.22467700000000002,0.07909600000000006,0.069944],[2,2,2,2,2,2,2],MutableBinaryHeap(),[0,1,2,5,3,6,4])

In [43]:
z.parents

7-element Array{Int64,1}:
 1
 7
 1
 6
 1
 7
 3

In [44]:
z.dists

7-element Array{Float64,1}:
 0.0     
 0.40426 
 0.047163
 0.346588
 0.224677
 0.079096
 0.069944

### Minimum Spanning Trees

In [45]:
E, W = kruskal_minimum_spantree(g, ww)

([edge [15]: 6 -- 7,edge [11]: 3 -- 7,edge [2]: 1 -- 3,edge [3]: 1 -- 5,edge [12]: 4 -- 6,edge [8]: 2 -- 7],[0.009152000000000049,0.02278100000000005,0.047162999999999955,0.22467700000000002,0.26749199999999995,0.33431599999999995])

In [46]:
mean(W)

0.15093016666666667

# I/O Operations with Graph Data

### Graph Transformation

In [47]:
gg = G2LG(g)

{7, 15} undirected graph

### Saving a Graph

In [48]:
save("graph data.jgz", gg, "AliceGang", compress=true)

1

### Loading a Graph

In [49]:
gg_loaded = load("graph data.jgz", "AliceGang")

{7, 15} undirected graph

### Graph Transformation Again

In [50]:
g_loaded = LG2G(gg_loaded)

Undirected Graph (7 vertices, 15 edges)

In [52]:
hcat(Graphs.edges(g_loaded), Graphs.edges(g))

15x2 Array{Graphs.Edge{Int64},2}:
 edge [1]: 1 -- 2   edge [1]: 1 -- 2 
 edge [2]: 1 -- 3   edge [2]: 1 -- 3 
 edge [3]: 1 -- 5   edge [3]: 1 -- 5 
 edge [4]: 1 -- 7   edge [4]: 1 -- 7 
 edge [5]: 2 -- 3   edge [5]: 2 -- 3 
 edge [6]: 2 -- 5   edge [6]: 2 -- 5 
 edge [7]: 2 -- 6   edge [7]: 2 -- 6 
 edge [8]: 2 -- 7   edge [8]: 2 -- 7 
 edge [9]: 3 -- 4   edge [9]: 3 -- 4 
 edge [10]: 3 -- 6  edge [10]: 3 -- 6
 edge [11]: 3 -- 7  edge [11]: 3 -- 7
 edge [12]: 4 -- 6  edge [12]: 4 -- 6
 edge [13]: 4 -- 7  edge [13]: 4 -- 7
 edge [14]: 5 -- 6  edge [14]: 5 -- 6
 edge [15]: 6 -- 7  edge [15]: 6 -- 7